 # A - Takoyaki

 https://atcoder.jp/contests/abc176/tasks/abc176_a

 ## 提出コード

In [ ]:
import math
n, x, t = [int(x) for x in input().split()]
print(int(math.ceil(n / x) * t))


 ## 公式解説

 ceilを使うのは言語によっては誤りになる（整数同士の除算で、結果がintに丸められてしまうので）．これを防ぐためには `math.ceil(n/x)` ではなく、`(n + x - 1) /x` のようにすると，ちょうど割り切れるとき以外は答えが1増えて良い．

In [ ]:
n = 12
x = 5
int((n + x - 1) / x)  # Pythonだと勝手に小数にしてくれてしまうので


 ## 公式解説を踏まえて訂正

In [ ]:
n, x, t = [int(x) for x in input().split()]
print(int((n + x - 1) / x) * t)


 # B - Multiple of 9

 https://atcoder.jp/contests/abc176/tasks/abc176_b

 ## 提出コード

In [ ]:
n_str = str(input())
x = 0
for e in n_str:
    x += int(e)
print('Yes') if x % 9 == 0 else print('No')


 ## 公式解説

 実はPythonのintは多倍長整数（可変長の配列で数値を表現するので、基本的に最大値がない）なので、いきなり`int(input()) % 9 == 0` としてもよかった．他の言語であれば上記の実装の通り．

 # C - Step

 https://atcoder.jp/contests/abc176/tasks/abc176_c

 ## 提出コード

In [ ]:
l = int(input())
xs = [int(x) for x in input().split()]
step = 0
max = 0
for x in xs:
    # This x must be the max in the preceeding numbers
    if x > max:
        max = x
        continue  # Current max, so no step is needed
    step += max - x
print(step)


 ## 公式解説

 基本的に上記の解き方で良いが，Python以外の言語の場合，stepの値は32bit intの最大値 2^31=2,147,483,648を超える可能性があるので気を付けよう．

 # D - Wizard in Maze

 https://atcoder.jp/contests/abc176/tasks/abc176_d

 ## コンテスト中の提出コード

 give up


 ## 公式解説

 以下，ワープ回数をコストと呼ぶ．
 まずスタート地点から幅優先探索を用いて，コスト0でたどり着けるマスを列挙する．
 次に，コスト0でたどり着けるマスから幅優先探索を用いて，コスト1でたどり着けて，かつコスト0でたどり着けないマスを列挙する．
 これを繰り返して，すべての到達可能なマスに対してコストを求める．

 このとき，コストが0と1だけの場合は01-BFSと呼ばれる実装が使える．（コスト0は両端キューの先頭，コスト1は終端に追加することで，距離順の探索が可能）
 公式解説には書いてなかったが，コストが0と1以外もある場合はPriority Queueというデータ構造が使えるらしい．

 各マスについて25マス分の移動しか調べないため，計算量はO(HW)．


 ## 公式解説を踏まえた実装（heapqを使う実装）

 PythonのPriority Queue実装であるheapqモジュールを使った実装．これだとPyPy3を選んでも，一つのテストケースで制限時間の2000msを超える．
 [提出結果（PyPy3）](https://atcoder.jp/contests/abc176/submissions/16174870)

 ちなみにPython3を選ぶと複数のテストケースでTLEになる．
 [提出結果（Python3）](https://atcoder.jp/contests/abc176/submissions/16175237)


In [ ]:
import heapq

H, W = [int(x) for x in input().split()]
c_h, c_w = [int(x) - 1 for x in input().split()]  # Convert to 0-origin
d_h, d_w = [int(x) - 1 for x in input().split()]  # Convert to 0-origin
s = [['.' for w in range(W)] for h in range(H)]
for h in range(H):
    s[h] = input()

def in_the_map(x):
    if x[0] < 0 or H <= x[0] or x[1] < 0 or W <= x[1]:
        return False
    return True

visited = [[False for w in range(W)] for h in range(H)]
queue = []
heapq.heappush(queue, (0, (c_h, c_w)))  # Push start point with cost (number of warps) 0

while len(queue) > 0:
    cost, p = heapq.heappop(queue)

    if visited[p[0]][p[1]]:  # Skip if already visited (the priority queue assures we can visit to the point with the smallest cost)
        continue
    visited[p[0]][p[1]] = True

    if p[0] == d_h and p[1] == d_w:  # Finally reach to the goal
        print(cost)
        break
    # print(f'Visit {p}: {cost}')

    # Explore neighbours where can be reached by walk (with the same cost as p)
    neighbours = ((p[0], p[1] - 1), (p[0], p[1] + 1), (p[0] - 1, p[1]), (p[0] + 1, p[1]))
    for n in neighbours:
        if not in_the_map(n) or visited[n[0]][n[1]]:
            continue
        if s[n[0]][n[1]] == '.':  # not wall
            heapq.heappush(queue, (cost, n))  # For later exploration

    # Explore points where can be reached by warp from visited points(with +1 cost from p)
    for diff_h in range(-2, 3, 1):
        for diff_w in range(-2, 3, 1):
            h = p[0] + diff_h
            w = p[1] + diff_w
            if not in_the_map((h, w)) or visited[h][w]:
                continue
            if s[h][w] == '.': # not wall
                heapq.heappush(queue, (cost + 1, (h, w)))

if not visited[d_h][d_w]:  # Cannot reach to the goal
    print(-1)



 ## 公式解説を踏まえた実装（Priority Queueを使わない版，AC）

 heapqの内部ソートが遅いと見込んで，Priority Queueを使わずに，単純にコストの小さいところを深さ優先探索（DFS）して，
 その後ワープ（+1のコスト）でいけるところをDFSする…という処理を繰り返すもの．これなら上記のコードの3分の1の実行時間で終了する．


In [ ]:
# 入力処理
H, W = [int(x) for x in input().split()]
c_h, c_w = [int(x) - 1 for x in input().split()]  # Convert to 0-origin
d_h, d_w = [int(x) - 1 for x in input().split()]  # Convert to 0-origin
s = [['.' for w in range(W)] for h in range(H)]
for h in range(H):
    s[h] = input()

def in_the_map(x):
    if x[0] < 0 or H <= x[0] or x[1] < 0 or W <= x[1]:
        return False
    return True
 
costs = [[-1 for w in range(W)] for h in range(H)]
 
def dfs(h, w, c, q):
    if not in_the_map((h, w)):
        return
    if costs[h][w] >= 0:
        return
    if s[h][w] != '.':
        return
    q.append((h, w))
    costs[h][w] = c 
    dfs(h - 1, w, c, q)
    dfs(h + 1, w, c, q)
    dfs(h, w - 1, c, q)
    dfs(h, w + 1, c, q)

queue = [(c_h, c_w)]
cost = 0
while len(queue) > 0:
    visited = []
    for q in queue:
        dfs(q[0], q[1], cost, visited)
    
    next_queue = []
    for v in visited:
        for dh in range(-2, 3):
            for dw in range(-2, 3):
                hh = v[0] + dh 
                ww = v[1] + dw
                if not in_the_map((hh, ww)):
                    continue
                if costs[hh][ww] >= 0:
                    continue
                if s[hh][ww] != '.':
                    continue
                next_queue.append((hh, ww))

    cost += 1
    queue = next_queue

print(costs[d_h][d_w])


 ## 他の人の提出コードを参考にした実装（dequeを利用）

 collections.dequeを使って公式解説で説明されていた01-BFSを実装する方法．
 [参考にしたコード](https://atcoder.jp/contests/abc176/submissions/16175747)
 [提出結果](https://atcoder.jp/contests/abc176/submissions/16176919)


In [ ]:
from collections import deque

H, W = [int(x) for x in input().split()]
c_h, c_w = [int(x) - 1 for x in input().split()]  # Convert to 0-origin
d_h, d_w = [int(x) - 1 for x in input().split()]  # Convert to 0-origin
s = [['.' for w in range(W)] for h in range(H)]
for h in range(H):
    s[h] = input()

def in_the_map(x):
    if x[0] < 0 or H <= x[0] or x[1] < 0 or W <= x[1]:
        return False
    return True

costs = [[-1 for w in range(W)] for h in range(H)]
queue = deque()
queue.append((c_h, c_w, 0))
while len(queue) > 0:
    q = queue.popleft()
    h = q[0]
    w = q[1]
    c = q[2]
    if costs[h][w] > -1:
        continue
    # print(f'{h}, {w}: {c}')
    costs[h][w] = c
    
    # Explore neighbours where can be visited with the same cost as this point
    neighbours = ((h, w - 1), (h, w + 1), (h - 1, w), (h + 1, w))
    for n in neighbours:
        if not in_the_map(n) or costs[n[0]][n[1]] > -1:  # 01-BFS assures that the cost we have recorded is the smallest
            continue
        if s[n[0]][n[1]] == '.':  # not wall
            queue.appendleft((n[0], n[1], c))  # This operation assures that the point with the lowest cost (+0) is popped earlier

    for dh in range(-2, 3):
        for dw in range(-2, 3):
            hh = h + dh 
            ww = w + dw
            if not in_the_map((hh, ww)) or costs[hh][ww] > -1:
                continue
            if s[hh][ww] == '.':
                queue.append((hh, ww, c + 1))

print(costs[d_h][d_w])


 ## 補足: PythonでのPriority Queueの使い方

 heapqが実装モジュール．[基本的な使用方法](https://docs.python.org/ja/3/library/heapq.html#basic-examples)

In [ ]:
import heapq
h = []
heapq.heappush(h, (1, (0, 2)))
heapq.heappush(h, (2, (1, 2)))
heapq.heappush(h, (0, (2, 2)))
for i in range(len(h)):
    print(heapq.heappop(h))


 queue.PriorityQueueもあるが，heapqよりも遅いという噂．ただしこちらはスレッドセーフ．
 [Queueオブジェクト共通のpublicメソッド](https://docs.python.org/ja/3/library/queue.html?highlight=priorityqueue#queue-objects)

In [ ]:
from queue import PriorityQueue
q = PriorityQueue()
q.put((1, (0, 2)))
q.put((2, (1, 2)))
q.put((0, (2, 2)))
for i in range(q.qsize()):
    print(q.get())
